In [ ]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

from herbie.scale import Scale
from herbie.generator import Generator
import herbie.filters as filters
from nb_utils import play, to_wav

# Dani California

Because if you don't know what to play, Dani California is always awesome.

Disclaimer: Chad's amazing drumming is gonna be missing until I figure out how to make drum sounds with sine waves.

## Bass

In [ ]:
bass = Generator()
bass.fn = lambda t: np.sin(t) + 0.1 * np.sin(2*t)
attack = (1, 1/20)
decay = (0.3, 1/8)
release = (0, 1/4)
adsr = lambda s: filters.adsr(s, attack, decay, release)
clip = lambda s: filters.clip(s, 0.96, 0.5)
bass.filters = [clip, adsr]

In [ ]:
am = Scale("A", "minor", octave=1, bpm=102)
am.default_note = 8
b1 = am.sequence("A A A^ r.16 Ev.16")
b2 = am.sequence("Gv Gv G F")
b3 = am.sequence("D D D^ r.16 D.16")
b4 = am.sequence("G.16 A^.16 G E D")
score = b1 + b2 + b3 + b4
bassline = bass.sequence(score)
bassline = np.tile(bassline, 3)
play(bassline, autoplay=False)

## Octave bass

In [ ]:
bass2 = Generator()
attack = (1, 1/20)
decay = (0.3, 1/8)
release = (0, 1/4)
adsr = lambda s: filters.adsr(s, attack, decay, release)
clip = lambda s: filters.clip(s, 0.5, 0.5)
bass2.filters = [clip, adsr]

In [ ]:
am = Scale("A", "minor", octave=2)
b1 = am.sequence("A A A^ r.8 Ev.8")
b2 = am.sequence("Gv Gv G F")
b3 = am.sequence("D D D^ r.8 D.8")
b4 = am.sequence("G.8 A^.8 G E D")
score = b1 + b2 + b3 + b4
bassline2 = bass2.sequence(score)
bassline2 = np.tile(bassline2, 3)
play(bassline2, autoplay=False)

## Rhythm Guitar

In [ ]:
rhy = Generator()
attack = (1, 1/16)
decay = (0.8, 1/8)
release = (0, 1/8)
adsr = lambda s: filters.adsr(s, attack, decay, release)
clip = lambda s: filters.clip(s, 0.8, 0.2)
rhy.filters = [clip, adsr]

In [ ]:
am = Scale("A", "minor", octave=4)
b1 = rhy.sequence([(am.chord(0)[:3], 2)])
b2 = rhy.sequence([(am.chord(6, -1)[:3], 2)])
b3 = rhy.sequence([(am.chord(4, -1)[:3], 2)])
b4 = rhy.sequence([(am.chord(4, -1)[1:3], 2)])
chords = np.concatenate([b1, b2, b3, b4])
chords = np.tile(chords, 3)
play(chords, autoplay=False)

## Mash em all together

In [ ]:
from typing import List
def masher(parts: List[np.array]) -> np.array:
    mlen = np.max([len(p) for p in parts])
    padded = []
    for p in parts:
        padded.append(np.pad(p, (0, mlen - len(p))))
    mashed = np.sum(padded, axis=0)
    return filters.normalize(mashed)

In [ ]:
song = np.array([masher([bassline*0.5, bassline2]), masher([bassline])])
play(song)

In [ ]:
to_wav("out/dani_california.wav", song.T)